# Custom Extensions with the Registry System

ReactorTwin uses a **plugin registry** system that makes it easy to add custom kinetics models,
reactor types, constraints, and more. This notebook shows how to:

1. Explore existing registries
2. Create a custom kinetics model
3. Create a custom reactor
4. Register and use custom components
5. Build a complete custom simulation

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from reactor_twin import (
    NeuralODE, PositivityConstraint, Trainer, ReactorDataGenerator,
)
from reactor_twin.utils import (
    Registry, KINETICS_REGISTRY, REACTOR_REGISTRY,
    CONSTRAINT_REGISTRY, NEURAL_DE_REGISTRY, DIGITAL_TWIN_REGISTRY,
)
from reactor_twin.reactors.kinetics.base import AbstractKinetics
from reactor_twin.reactors.base import AbstractReactor

np.random.seed(42)
torch.manual_seed(42)

## 1. Exploring the Registries

ReactorTwin has 5 registries, one for each major component type.

In [ ]:
registries = {
    "Kinetics": KINETICS_REGISTRY,
    "Reactors": REACTOR_REGISTRY,
    "Constraints": CONSTRAINT_REGISTRY,
    "Neural DEs": NEURAL_DE_REGISTRY,
    "Digital Twin": DIGITAL_TWIN_REGISTRY,
}

for name, reg in registries.items():
    keys = reg.list_keys()
    print(f"\n{name} Registry ({len(keys)} items):")
    for key in keys:
        cls = reg.get(key)
        print(f"  {key:30s} -> {cls.__name__}")

## 2. Creating a Custom Kinetics Model

Let's create a **Competitive Inhibition** kinetics model (common in enzyme reactions):

$$r = \frac{V_{max} \cdot [S]}{K_m(1 + [I]/K_i) + [S]}$$

where $[S]$ is substrate, $[I]$ is inhibitor, $V_{max}$ is max rate, $K_m$ is Michaelis constant, and $K_i$ is inhibition constant.

In [ ]:
class CompetitiveInhibitionKinetics(AbstractKinetics):
    """Competitive inhibition enzyme kinetics.
    
    Species order: [Substrate, Product, Inhibitor]
    Reaction: S -> P (inhibited by I)
    """
    
    def __init__(self, name: str, params: dict):
        super().__init__(
            name=name,
            num_reactions=1,
            params=params,
        )
        self.Vmax = params["Vmax"]
        self.Km = params["Km"]
        self.Ki = params["Ki"]
    
    def compute_rates(self, concentrations, temperature):
        S = max(concentrations[0], 0.0)
        I = max(concentrations[2], 0.0)  # Inhibitor
        
        # Competitive inhibition rate
        rate = self.Vmax * S / (self.Km * (1 + I / self.Ki) + S)
        
        # S -> P, I unchanged
        return np.array([-rate, rate, 0.0])


# Test the kinetics
kinetics = CompetitiveInhibitionKinetics(
    name="competitive_inhibition",
    params={"Vmax": 1.0, "Km": 0.5, "Ki": 0.1},
)

# Rate at different inhibitor concentrations
S_test = 1.0
for I_conc in [0.0, 0.1, 0.5, 1.0, 5.0]:
    rates = kinetics.compute_rates([S_test, 0.0, I_conc], 300.0)
    print(f"  [I] = {I_conc:.1f}: rate = {-rates[0]:.4f} (inhibition = {1 - (-rates[0] / 0.6667):.1%})")

## 3. Registering Custom Components

Register the custom kinetics so it can be discovered by key.

In [ ]:
# Register our custom kinetics
KINETICS_REGISTRY.register("competitive_inhibition", CompetitiveInhibitionKinetics)

# Verify it's registered
print("Updated Kinetics Registry:")
for key in KINETICS_REGISTRY.list_keys():
    print(f"  {key}")

# Retrieve by key
KineticsClass = KINETICS_REGISTRY.get("competitive_inhibition")
print(f"\nRetrieved: {KineticsClass.__name__}")

## 4. Using Custom Kinetics in a Reactor

Let's simulate a batch reactor with competitive inhibition.

In [ ]:
from reactor_twin import BatchReactor

# Create batch reactor with competitive inhibition
reactor = BatchReactor(
    name="enzyme_batch",
    num_species=3,  # S, P, I
    params={
        "V": 1.0,
        "T": 310.0,  # 37°C (body temperature)
        "C_initial": [1.0, 0.0, 0.0],  # Start with substrate only
    },
    kinetics=kinetics,
    isothermal=True,
)

# Simulate without inhibitor
y0 = reactor.get_initial_state()
t_eval = np.linspace(0, 10, 100)
sol_no_inhib = solve_ivp(reactor.ode_rhs, [0, 10], y0, t_eval=t_eval, method="LSODA")

# Simulate with inhibitor
reactor_inhib = BatchReactor(
    name="enzyme_batch_inhibited",
    num_species=3,
    params={"V": 1.0, "T": 310.0, "C_initial": [1.0, 0.0, 0.5]},  # Add inhibitor
    kinetics=kinetics,
    isothermal=True,
)
y0_inhib = reactor_inhib.get_initial_state()
sol_inhib = solve_ivp(reactor_inhib.ode_rhs, [0, 10], y0_inhib, t_eval=t_eval, method="LSODA")

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(t_eval, sol_no_inhib.y[0], 'b-', label='Substrate')
axes[0].plot(t_eval, sol_no_inhib.y[1], 'r-', label='Product')
axes[0].set_title('Without Inhibitor')
axes[0].set_xlabel('Time'); axes[0].set_ylabel('Concentration')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(t_eval, sol_inhib.y[0], 'b-', label='Substrate')
axes[1].plot(t_eval, sol_inhib.y[1], 'r-', label='Product')
axes[1].plot(t_eval, sol_inhib.y[2], 'g--', label='Inhibitor')
axes[1].set_title('With Inhibitor ([I]=0.5)')
axes[1].set_xlabel('Time'); axes[1].set_ylabel('Concentration')
axes[1].legend(); axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Without inhibitor - 50% conversion time: ~{t_eval[np.argmin(np.abs(sol_no_inhib.y[0] - 0.5))]:.1f} min")
print(f"With inhibitor    - 50% conversion time: ~{t_eval[np.argmin(np.abs(sol_inhib.y[0] - 0.5))]:.1f} min")

## 5. Training a Neural ODE on Custom Kinetics

Learn the inhibition dynamics with a Neural ODE.

In [ ]:
# Prepare training data (uninhibited case)
z0_torch = torch.tensor(y0, dtype=torch.float32).unsqueeze(0)
t_torch = torch.tensor(t_eval, dtype=torch.float32)
targets_torch = torch.tensor(sol_no_inhib.y.T, dtype=torch.float32).unsqueeze(0)

# Create and train model
model = NeuralODE(
    state_dim=3,  # 3 species
    hidden_dim=64,
    num_layers=3,
    solver="rk4",
    adjoint=False,
)

# Apply positivity constraint during training
constraint = PositivityConstraint(mode="hard", method="softplus")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

losses = []
model.train()
for epoch in range(300):
    optimizer.zero_grad()
    preds = model(z0_torch, t_torch)
    preds_constrained, _ = constraint(preds)
    loss_dict = model.compute_loss(preds_constrained, targets_torch)
    loss_dict["total"].backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    losses.append(loss_dict["total"].item())
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}: loss = {losses[-1]:.6f}")

# Evaluate
model.eval()
with torch.no_grad():
    preds = model(z0_torch, t_torch)
    preds_constrained, _ = constraint(preds)

pred_np = preds_constrained[0].numpy()
true_np = sol_no_inhib.y.T

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(t_eval, true_np[:, 0], 'b--', linewidth=2, label='True S')
ax.plot(t_eval, true_np[:, 1], 'r--', linewidth=2, label='True P')
ax.plot(t_eval, pred_np[:, 0], 'b-', label='Pred S')
ax.plot(t_eval, pred_np[:, 1], 'r-', label='Pred P')
ax.set_xlabel('Time'); ax.set_ylabel('Concentration')
ax.set_title('Neural ODE Learning Custom Kinetics')
ax.legend(); ax.grid(True, alpha=0.3)
plt.show()

print(f"MSE: {np.mean((pred_np - true_np)**2):.6f}")

## 6. Creating a Custom Registry

You can create your own registries for any component type.

In [ ]:
# Create a custom registry
OPTIMIZER_REGISTRY = Registry("optimizers")

# Register some optimizers
OPTIMIZER_REGISTRY.register("adam", torch.optim.Adam)
OPTIMIZER_REGISTRY.register("sgd", torch.optim.SGD)
OPTIMIZER_REGISTRY.register("lbfgs", torch.optim.LBFGS)

print(f"Available optimizers: {OPTIMIZER_REGISTRY.list_keys()}")

# Use it
OptimizerClass = OPTIMIZER_REGISTRY.get("adam")
opt = OptimizerClass(model.parameters(), lr=1e-3)
print(f"Created optimizer: {type(opt).__name__}")

## 7. Using the Data Generator with Custom Reactors

In [ ]:
# Use ReactorDataGenerator with the custom reactor
data_gen = ReactorDataGenerator(reactor=reactor)

z0_batch, t_batch, targets_batch = data_gen.generate(
    num_trajectories=10,
    t_span=(0, 8),
    num_points=50,
    noise_std=0.01,
)

print(f"Generated data:")
print(f"  Initial conditions: {z0_batch.shape}")
print(f"  Time points:        {t_batch.shape}")
print(f"  Target trajectories: {targets_batch.shape}")

# Visualize a few trajectories
fig, ax = plt.subplots(figsize=(10, 5))
for i in range(min(5, z0_batch.shape[0])):
    ax.plot(t_batch.numpy(), targets_batch[i, :, 0].numpy(), alpha=0.5)
ax.set_xlabel('Time'); ax.set_ylabel('Substrate Concentration')
ax.set_title('Generated Training Trajectories (Substrate)')
ax.grid(True, alpha=0.3)
plt.show()

## Summary

| Feature | How |
|---------|-----|
| Explore registries | `KINETICS_REGISTRY.list_keys()` |
| Custom kinetics | Subclass `AbstractKinetics`, implement `compute_rates()` |
| Custom reactor | Subclass `AbstractReactor`, implement `ode_rhs()` |
| Register components | `REGISTRY.register(key, Class)` |
| Create new registries | `Registry(name)` |
| Generate data | `ReactorDataGenerator(reactor).generate(...)` |

The registry system makes ReactorTwin fully extensible — you can add new kinetics, reactors,
constraints, or Neural DE variants without modifying any core code.